In [2]:
import io
import os
import shutil
import json
import random
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import keras
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [8]:
patha = os.getcwd()

image_dir = os.path.join(patha+'/512px/')
metadata = os.path.join(patha+'/metadata/')

list_of_jsons = os.listdir(metadata)

# storage_dir = 'D:/UMSI Classes/Fall2018/670/Project/danbooru2017/Selected/'
# test_dir = 'D:/UMSI Classes/Fall2018/670/Project/danbooru2017/Test/'

storage_dir2 = os.path.join(patha+'/Final model/Training/')
test_dir2 = os.path.join(patha+'/Final model/Test/')
os.mkdir(test_dir2)
os.mkdir(storage_dir2)

In [47]:
#Get all potential filenames (JSON includes all files in dataset, only have storage for a portion)
all_img_files = []

for folder in os.listdir(image_dir):
    imgfolder = os.path.join(image_dir+folder)
    all_img_files += [imeg.replace('.jpg', '') for imeg in os.listdir(imgfolder)]

In [9]:
all_json_ids = []

for metadata_json in list_of_jsons:
    json_path = os.path.join(metadata+metadata_json)
    print('Running on json file {}'.format(metadata_json))
    with open(json_path, 'r', encoding='utf-8') as file:
        for num, line in enumerate(file):
            try:
                json_set = json.loads(line)
                all_json_ids.append(json_set['id'])
            except Exception:
                print('No')

Running on json file 2017000000000000.json
Running on json file 2017000000000001.json
Running on json file 2017000000000002.json
Running on json file 2017000000000003.json
Running on json file 2017000000000004.json
Running on json file 2017000000000005.json
Running on json file 2017000000000006.json
Running on json file 2017000000000007.json
Running on json file 2017000000000008.json
Running on json file 2017000000000009.json
Running on json file 2017000000000010.json
Running on json file 2017000000000011.json
Running on json file 2017000000000012.json
Running on json file 2017000000000013.json
Running on json file 2017000000000014.json
Running on json file 2017000000000015.json
Running on json file 2017000000000016.json


In [10]:
def image_select(line, taglist):
    tag_vector = np.zeros(5)
    json_set = json.loads(line)
    taglist_s = set(taglist)

    xx = set([x['name'] for x in json_set['tags']])
    tag_target = xx.intersection(taglist_s)
    if tag_target:
        for number in range(len(taglist)):
            if taglist[number] in tag_target:
                tag_vector[number] = 1
    return (int(json_set['id']), tag_vector)

In [26]:
#Copy picture to new directory function
def copy_function(target_image_list, destination):
    print('Copying from 512px to {}'.format(destination))
    
    not_found = []
    for image in target_image_list:
        id_ = int(image[0])
        imagefolder = str(id_%1000)
        imagefolder = '0' * (4-len(imagefolder)) + imagefolder
        filename = os.path.join(image_dir+imagefolder+'/'+str(id_)+'.jpg')
        try:      
            shutil.copy2(filename, destination)
        except Exception:
            not_found.append(image)
            print('{} not found'.format(filename))
            continue
    
    cleaned_list = [x for x in target_image_list if x not in not_found]
    
    print('Completed')
    return cleaned_list

#### Make list of which pictures to copy

In [11]:
taglist = ['1girl', 'long_hair', 'blush', 'smile', 'short_hair']

In [45]:
# '''
# Reservoir sampling >> Find pictures with said tags, so long as they are in the SFW set (JSON file may include things that
#   are NSFW)
# '''
# i = 0
# target_images = []
# test_images = []

# list_of_jsons = os.listdir(metadata)

# for metadata_json in list_of_jsons:
#     json_path = os.path.join(metadata+metadata_json)
#     print('Running on json file {}'.format(metadata_json))
#     with open(json_path, 'r', encoding='utf-8') as file:
#         random.seed(453432)
#         for num, line in enumerate(file):
#             try:
#                 json_set = json.loads(line)
#                 #check if id exists and is safe for work
#                 if int(json_set['id'])%1000 >= 100 or json_set['rating'] != 's':
#                     continue

#                 if len(target_images) < 5000:
#                     target_images.append(image_select(line, taglist))
#                 elif len(test_images) < 500:
#                     test_images.append(image_select(line, taglist))
#                 else: 
#                     randsel = random.randint(0, num)
#                     if randsel < 5000:
#                         target_images[randsel] = image_select(line, taglist)

#                 if i%10000 == 0:
#                     print(i)

#                 i+= 1
#             except Exception:
#                 print(line)

Running on json file 2017000000000000.json
0
10000
Running on json file 2017000000000001.json
20000
Running on json file 2017000000000002.json
30000
Running on json file 2017000000000003.json
40000
50000
Running on json file 2017000000000004.json
60000
Running on json file 2017000000000005.json
70000
Running on json file 2017000000000006.json
80000
90000
Running on json file 2017000000000007.json
100000
Running on json file 2017000000000008.json
110000
Running on json file 2017000000000009.json
120000
130000
Running on json file 2017000000000010.json
140000
Running on json file 2017000000000011.json
150000
Running on json file 2017000000000012.json
160000
170000
Running on json file 2017000000000013.json
180000
Running on json file 2017000000000014.json
190000
Running on json file 2017000000000015.json
200000
210000
Running on json file 2017000000000016.json
220000


In [25]:
'''
Reservoir sampling >> 1500 for each class, and 500 negative samples
'''
i = 0
target_images2 = []
test_images2 = []
c1 = 0
c2 = 0
c3 = 0
c4 = 0
c5 = 0
cass = 0
cneg = 0

list_of_jsons = os.listdir(metadata)

for metadata_json in list_of_jsons:
    json_path = os.path.join(metadata+metadata_json)
    print('Running on json file {}'.format(metadata_json))
    with open(json_path, 'r', encoding='utf-8') as file:
        random.seed(453432)
        for num, line in enumerate(file):
#             try:
                if len(target_images2) == 10000 and len(test_images2) == 1000:
                    break
                
                json_set = json.loads(line)
                #check if id exists and is safe for work
                if int(json_set['id'])%1000 >= 100 or json_set['rating'] != 's':
                    continue
                
                randsel = random.randint(1, 3)
                if randsel == 3:
                    continue
                
                img = image_select(line, taglist)
                
                if len(target_images2) < 10000:
                    if c1 < 1500 and img[1][0] == 1 and sum(img[1]) == 1:
                        c1 += 1
                        target_images2.append(image_select(line, taglist))
                    elif c2 < 1500 and img[1][1] == 1  and sum(img[1]) == 1:
                        c2 += 1
                        target_images2.append(image_select(line, taglist))
                    elif c3 < 1500 and img[1][2] == 1  and sum(img[1]) == 1:
                        c3 += 1
                        target_images2.append(image_select(line, taglist))
                    elif c4 < 1500 and img[1][3] == 1  and sum(img[1]) == 1:
                        c4 += 1
                        target_images2.append(image_select(line, taglist))
                    elif c5 < 1500 and img[1][4] == 1  and sum(img[1]) == 1:
                        c5 += 1
                        target_images2.append(image_select(line, taglist))
                    elif cass < 2000 and sum(img[1]) > 1:
                        cass += 1
                        target_images2.append(image_select(line, taglist))
                    elif cneg < 500 and sum(img[1]) == 0:
                        cneg += 1
                        target_images2.append(image_select(line, taglist))
                elif len(test_images2) < 1000:
                    test_images2.append(image_select(line, taglist))

                if i%10000 == 0:
                    print((c1,c2,c3,c4,c5,cneg), i)

                i+= 1
                
                
                    
#             except Exception:
#                 print(line)

Running on json file 2017000000000000.json
(0, 0, 0, 1, 0, 0) 0
Running on json file 2017000000000001.json
(1300, 378, 137, 257, 241, 500) 10000
Running on json file 2017000000000002.json
(1500, 753, 267, 528, 472, 500) 20000
Running on json file 2017000000000003.json
(1500, 1096, 416, 764, 686, 500) 30000
Running on json file 2017000000000004.json
(1500, 1465, 529, 993, 914, 500) 40000
Running on json file 2017000000000005.json
(1500, 1500, 652, 1220, 1144, 500) 50000
Running on json file 2017000000000006.json
(1500, 1500, 773, 1456, 1371, 500) 60000
Running on json file 2017000000000007.json
(1500, 1500, 893, 1500, 1500, 500) 70000
Running on json file 2017000000000008.json
Running on json file 2017000000000009.json
(1500, 1500, 1002, 1500, 1500, 500) 80000
Running on json file 2017000000000010.json
(1500, 1500, 1112, 1500, 1500, 500) 90000
Running on json file 2017000000000011.json
(1500, 1500, 1228, 1500, 1500, 500) 100000
Running on json file 2017000000000012.json
(1500, 1500, 134

#### Copies pics from folder 1 to 2

In [57]:
#Copies sampled pictures into folder

# target_ids = [int(x[0]) for x in target_images]
# test_ids = [int(x[0]) for x in test_images]
# not_found = []

# training_final = copy_function(target_images, storage_dir)
# test_final = copy_function(test_images, test_dir)

# for image in target_images:
#     id_ = int(image[0])
#     imagefolder = str(id_%1000)
#     imagefolder = '0' * (4-len(imagefolder)) + imagefolder
#     filename = os.path.join(image_dir+imagefolder+'/'+str(id_)+'.jpg')
#     try:      
#         shutil.copy2(filename, storage_dir)
#     except Exception:
#         not_found.append(image)
#         print(filename)
#         continue

# for image in test_images:
#     id_ = int(image[0])
#     imagefolder = str(id_%1000)
#     imagefolder = '0' * (4-len(imagefolder)) + imagefolder
#     filename = os.path.join(test_dir+imagefolder+'/'+str(id_)+'.jpg')
#     try:      
#         shutil.copy2(filename, storage_dir)
#     except Exception:
#         not_found.append(image)
#         print(filename)
#         continue

Copying from 512px to D:/UMSI Classes/Fall2018/670/Project/danbooru2017/Selected/
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0007/2980007.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0033/104033.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0025/2991025.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0014/85014.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0049/2994049.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0030/2996030.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0097/2989097.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0080/1002080.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0014/2979014.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0041/2203041.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0088/2990088.jpg not found
D:/UMSI 

In [29]:
#Copies sampled pictures into folder

target_ids = [int(x[0]) for x in target_images2]
test_ids = [int(x[0]) for x in test_images2]

training_final_v2 = copy_function(target_images2, storage_dir2)
test_final_v2 = copy_function(test_images2, test_dir2)

# for image in target_images:
#     id_ = int(image[0])
#     imagefolder = str(id_%1000)
#     imagefolder = '0' * (4-len(imagefolder)) + imagefolder
#     filename = os.path.join(image_dir+imagefolder+'/'+str(id_)+'.jpg')
#     try:      
#         shutil.copy2(filename, storage_dir)
#     except Exception:
#         not_found.append(image)
#         print(filename)
#         continue

# for image in test_images:
#     id_ = int(image[0])
#     imagefolder = str(id_%1000)
#     imagefolder = '0' * (4-len(imagefolder)) + imagefolder
#     filename = os.path.join(test_dir+imagefolder+'/'+str(id_)+'.jpg')
#     try:      
#         shutil.copy2(filename, storage_dir)
#     except Exception:
#         not_found.append(image)
#         print(filename)
#         continue

Copying from 512px to D:/UMSI Classes/Fall2018/670/Project/danbooru2017/Final model/Training/
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0004/33004.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0025/2991025.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0084/2990084.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0024/2024.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0013/487013.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0062/2983062.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0041/2203041.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0011/2983011.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0017/2982017.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0009/441009.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0076/298076.jpg not found
D

D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0033/2981033.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0012/2981012.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0038/2990038.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0061/2996061.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0097/327097.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0099/636099.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0099/2246099.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0090/2983090.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0092/2979092.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0012/614012.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0031/2977031.jpg not found
D:/UMSI Classes/Fall2018/670/Project/danbooru2017/512px/0074/2946074.jpg not found
D:/UMSI

In [60]:
# with open('training.pkl', 'wb') as file:
#     pickle.dump(training_final, file)
# with open('test.pkl', 'wb') as file:
#     pickle.dump(test_final, file)

In [31]:
with open('trainingv2.pkl', 'wb') as file:
    pickle.dump(training_final_v2, file)
with open('testv2.pkl', 'wb') as file:
    pickle.dump(test_final_v2, file)